# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [27]:
import os
import shutil
from azureml.core.dataset import Dataset
import joblib
from azureml.core import Workspace, Experiment, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice.aci import AciWebservice
from azureml.core.webservice import Webservice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [28]:
ws = Workspace.from_config()
experiment_name = 'Framingham-Experiment'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-141869
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-141869


In [29]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "Compute-Cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [30]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
from azureml.core.dataset import Dataset
found = False
key = "Framingham"
description_text = "Frahmingham Heart Disease Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/Haris-09/Coronary-Heart-Disease-Prediction-Capstone-Project/master/starter_file/framingham.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

In [31]:
df = dataset.to_pandas_dataframe()
df.describe()

,male,age,currentSmoker,prevalentStroke,prevalentHyp,diabetes,sysBP,diaBP,heartRate,TenYearCHD
count,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4239.000000,4240.000000
mean,0.429245,49.580189,0.494104,0.005896,0.310613,0.025708,132.354599,82.897759,75.878981,0.151887
std,0.495027,8.572942,0.500024,0.076569,0.462799,0.158280,22.033300,11.910394,12.025348,0.358953
min,0.000000,32.000000,0.000000,0.000000,0.000000,0.000000,83.500000,48.000000,44.000000,0.000000
25%,0.000000,42.000000,0.000000,0.000000,0.000000,0.000000,117.000000,75.000000,68.000000,0.000000
50%,0.000000,49.000000,0.000000,0.000000,0.000000,0.000000,128.000000,82.000000,75.000000,0.000000
75%,1.000000,56.000000,1.000000,0.000000,1.000000,0.000000,144.000000,90.000000,83.000000,0.000000
max,1.000000,70.000000,1.000000,1.000000,1.000000,1.000000,295.000000,142.500000,143.000000,1.000000


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [32]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.02, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = {
        "--C" :        choice(0.002, 0.02, 0.2, 2.0),
        "--max_iter" : choice(100, 200, 300, 500)
}
ps = RandomParameterSampling(param_sampling)

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory = "./", entry_script="train.py", compute_target=compute_target, script_params=None)

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                    primary_metric_name='Accuracy', 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    policy=early_termination_policy,
                                    max_total_runs=20,
                                    max_concurrent_runs=4,
                                    estimator=estimator
                                   )

In [33]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [34]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [22]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print(' Accuracy:', best_run_metrics['Accuracy'])
print(' Regularization Strength:',best_run_metrics['Regularization Strength:'])
print(' Max Iterations:',best_run_metrics['Max iterations:'])


AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [23]:
#TODO: Save the best model
best_model = best_run.register_model(model_name='Best_HyperDrive', model_path='./', 
                                properties={'Accuracy': best_run_metrics['Accuracy'],
                                            'Regularization Strength': best_run_metrics['Regularization Strength:'],
                                           'Max Iterations': best_run_metrics['Max iterations:']})

best_run.download_file('./model.joblib', 'model.joblib')

AttributeError: 'NoneType' object has no attribute 'register_model'

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service